In [ ]:
import pandas as pd

def question1(df):
    df_2021 = df[df["year"] == 2021]
    grouped = df_2021.groupby("pc_left").agg(
        median_price=("price", "median"),
        txn_count=("price", "count")
    )
    filtered = grouped[grouped["txn_count"] > 10]
    result = filtered.nsmallest(10, "median_price")
    result.to_csv("results/q1_top_10_lowest_median_2021.csv")

def question2(df):
    df_cb = df[(df["year"] == 2021) & (df["pc_left"].str.startswith("CB"))]
    grouped = df_cb.groupby("pc_left").agg(
        mean_price=("price", "mean"),
        txn_count=("price", "count")
    )
    filtered = grouped[grouped["txn_count"] > 10]
    result = filtered.nsmallest(5, "mean_price")
    result.to_csv("results/q2_cb_top5_lowest_mean_2021.csv")

def compute_cumulative_diff(df, metric):
    yearly = df.groupby(["pc_left", "year"]).agg(
        val=(metric, "median" if metric == "price" else "mean"),
        count=(metric, "count")
    ).reset_index()
    yearly.sort_values(["pc_left", "year"], inplace=True)
    yearly["diff"] = yearly.groupby("pc_left")["val"].diff().fillna(0)
    cumulative = yearly.groupby("pc_left").agg(
        cumulative_diff=("diff", "sum"),
        total_txns=("count", "sum")
    )
    return cumulative

def question3(df):
    cumulative = compute_cumulative_diff(df, metric="price")
    filtered = cumulative[cumulative["total_txns"] > 10]
    result = filtered.nsmallest(25, "cumulative_diff")
    result.to_csv("results/q3_top25_lowest_cumulative_median_diff.csv")

def question4(df):
    df_cb = df[df["pc_left"].str.startswith("CB")]
    yearly = df_cb[df_cb["year"] == 2021]
    grouped = yearly.groupby("pc_left").agg(
        median_price=("price", "median"),
        txn_count=("price", "count")
    )
    filtered = grouped[grouped["txn_count"] > 10]
    top5 = filtered.nsmallest(5, "median_price").index.tolist()

    cumulative = compute_cumulative_diff(df_cb, metric="price")
    result = cumulative.loc[top5].sort_values("cumulative_diff")
    result.to_csv("results/q4_cb_top5_sorted_by_cumulative_mean_diff.csv")

In [3]:
import utils as ut
df = ut.load_all_data()
question1(df)
question2(df)
question3(df)
question4(df)

KeyboardInterrupt: 